<a href="https://colab.research.google.com/github/roco999999/deneme/blob/main/euasproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install resampy

In [ ]:
!pip install librosa

In [ ]:
import pandas as pd
import resampy
import os
import librosa
import librosa.display
import numpy as np
import IPython.display as ipd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from tqdm import tqdm

In [ ]:
from google.colab import files

# Kaggle.json dosyasını seçin ve yükleyin  "C:\Users\oguzd\Desktop\audio-classification\kaggle.json"
uploaded = files.upload()

In [ ]:
!pwd

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [ ]:
!kaggle datasets download -d chrisfilo/urbansound8k

In [ ]:
!unzip urbansound8k.zip

In [ ]:
file_name='fold5/100263-2-0-121.wav'

audio_data, sampling_rate = librosa.load(file_name)
librosa.display.waveshow(audio_data,sr=sampling_rate)
ipd.Audio(file_name)


In [ ]:
audio_data


In [ ]:
sampling_rate

In [ ]:
audio_dataset_path='/content/'
metadata=pd.read_csv('UrbanSound8K.csv')
metadata.head(10)

In [ ]:
metadata['class'].value_counts()

In [ ]:
mfccs = librosa.feature.mfcc(y=audio_data, sr=sampling_rate, n_mfcc=40)

In [ ]:
mfccs

In [ ]:
import librosa
import numpy as np

def features_extractor(file):
    audio, sample_rate = librosa.load(file, sr=None)
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)

    return mfccs_scaled_features

In [ ]:
import resampy

extracted_features = []

for index_num, row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path), 'fold' + str(row["fold"]) + '/', str(row["slice_file_name"]))
    final_class_labels = row["class"]
    data = features_extractor(file_name)
    extracted_features.append([data, final_class_labels])

In [ ]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head(10)

In [ ]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [ ]:
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
num_labels = 10

model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

num_epochs = 200
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5',
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

In [ ]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

In [ ]:
import librosa
import numpy as np


filename = "fold5/6508-9-1-0.wav"

# Ses dosyasını yükle
audio, sample_rate = librosa.load(filename, sr=None)

# Yükseklik değerlerini normalize et
audio = audio / np.max(np.abs(audio))

# Eğer sample_rate'yi almak istiyorsanız:
print("Sample Rate:", sample_rate)

# MFCC özelliklerini çıkar
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)

#print(mfccs_scaled_features)
mfccs_scaled_features = mfccs_scaled_features.reshape(1, -1)
#print(mfccs_scaled_features)
#print(mfccs_scaled_features.shape)

# Tahmin yap
predicted_label = model.predict(mfccs_scaled_features)
#print(predicted_label)

# Tahmin sonucunu etiket olarak çevir
classes_x = np.argmax(predicted_label, axis=1)
prediction_class = labelencoder.inverse_transform(classes_x)
print(prediction_class)


In [ ]:
metadata['class'].value_counts()